In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from model import *
from model import Model_Rescorla_Wagner


In [ ]:
# -------------------
# Blocking Design 
# -------------------
# Define the model
experiment = Model_Rescorla_Wager(experiment_name="Blocking", lambda_US=1, beta_US=0.5)

# Define the predictors
A = Predictor(name='A', alpha = 0.2)
B = Predictor(name='B',alpha = 0.2)
C = Predictor(name='C',alpha = 0.2)

# Define the experiment groups
experiment_group = Group(name="Experiment Group")
experiment_group.add_phase_for_group(phase_name='Conditioning', predictors=[A], outcome=True, number_of_trial=10)
experiment_group.add_phase_for_group(phase_name='Blocking', predictors=[A], outcome=True, number_of_trial=10)
experiment.add_group(experiment_group)

control_group = Group(name="Control Group")
control_group.add_phase_for_group(phase_name='Conditioning', predictors=[C], outcome=True, number_of_trial=10)
control_group.add_phase_for_group(phase_name='Blocking', predictors=[A,B], outcome=True, number_of_trial=10)
experiment.add_group(control_group)

# Run the model
experiment.model_run()
experiment.display_results(save_to_file=True)

In [ ]:
experiment.result.head()

In [ ]:
# -------------------
# Extinction Design
# -------------------

# Define the model
exitinction = Model_Rescorla_Wager(experiment_name="Extinction", lambda_US=1, beta_US=0.5)

# Define the predictors
A = Predictor(name='A', alpha = 0.2)
# Define the experiment groups
exitinction_group = Group(name="Experiment Group")
exitinction_group.add_phase_for_group(phase_name='Conditioning', predictors=[A], outcome=True, number_of_trial=10)
exitinction_group.add_phase_for_group(phase_name='Extinction', predictors=[A], outcome=False, number_of_trial=10)
exitinction.add_group(exitinction_group)

# Run the model
exitinction.model_run()
exitinction.display_results(save_to_file=True)